## Langchain라이브러리에서 청킹(chunking)을 지원해주는 클래스들을 활용해보자
1. characterTextSplitter
   - 텍스트를 지정된 기준으로 나눠주는 기본적인 클래스
   - 텍스트가 간단한 구조일 때 적절(잘 정돈된 텍스트, csv파일 등)
   - 구분자는 디폴트로 "\n\n"을 사용하며 구분자가 단일 문자 이기 때문에 연속된 텍스트의 의미를 충분히 고려하지 못하고 청킹을 진행해버릴 수 있음
2. RecursiveCharacterTextSplitter
   - 텍스트를 여러 단계를 거쳐 나누어주는 클래스(청킹에서 가장 많이 활용됨)
   - 텍스트가 상대적으로 복잡한 구조일 때 적절(일반적인 문서, 웹페이지 데이터, 보고서 등)
   - 리스트를 통해 구분자를 여러 개 사용하여 세분화된 청크로 만들 수 있음
   - 처음엔 "\n\n"으로 분리하고 청크가 여전히 크다면 "\n", 그래도 크다면 ""이런 식으로 지정된 크기 제한 내에 들어올 때까지 계층적으로 분할을 진행

In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [8]:
text1 = "청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?"
text2 = "청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다."
text3 = """청킹을 위한 예제 데이터입니다. 
어떻게 쪼개지는지 알아볼까요?

결과를 확인해봅시다.
"""

### 1) Character Splitter

In [9]:
# 청킹(splitter) 객체 생성
chunk_size = 20   # 하나의 청크에 대한 길이 기준을 문자 20개로 설정
chunk_overlap = 0

c_splitter = CharacterTextSplitter(
    # 각 청크의 크기 지정
     # 의미있는 문맥을 유지하기 위해 일반적인 문서에서는 200~1500사이로 지정
     # (데이터의 종류에 따라서 달라지며 기술 문서나 연구 논문은 상대적으로 청크를 크게 지정하고 뉴스기사나 일반 문서들을 상대적으로 작게 지정)
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [11]:
# split_text : 청킹 실행
c_splitter.split_text(text1)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?']

In [12]:
c_splitter.split_text(text2)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다.']

In [13]:
c_splitter.split_text(text3)

Created a chunk of size 35, which is longer than the specified 20


['청킹을 위한 예제 데이터입니다. \n어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

### 2) Recursive Splitter

In [15]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [16]:
r_splitter.split_text(text1)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?']

In [17]:
r_splitter.split_text(text2)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

In [18]:
r_splitter.split_text(text3)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

### Character, Recursive 비교

In [19]:
see_you_again = """It's been a long day without you, my friend.
And I'll tell you all about it when I see you again.
We've come a long way from where we began.
Oh, I'll tell you all about it when I see you again.
When I see you again.


Damn, who knew?
All the planes we flew, good things we been through.
That I'd be standing right here talking to you.
'Bout another path, I know we loved to hit the road and laugh.
But something told me that it wouldn't last.
Had to switch up, look at things different, see the bigger picture.
Those were the days, hard work forever pays.
Now I see you in a better place (see you in a better place).
Uh."""

In [20]:
len(see_you_again)

620

In [21]:
c_splitter = CharacterTextSplitter(chunk_size=100,
                                   chunk_overlap=0,
                                   separator="\n\n"  # 구분자 직접 지정 가능(디폴트는 \n\n)
                                  )

c_splitter.split_text(see_you_again)

Created a chunk of size 215, which is longer than the specified 100


["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.\nWe've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.\nWhen I see you again.",
 "Damn, who knew?\nAll the planes we flew, good things we been through.\nThat I'd be standing right here talking to you.\n'Bout another path, I know we loved to hit the road and laugh.\nBut something told me that it wouldn't last.\nHad to switch up, look at things different, see the bigger picture.\nThose were the days, hard work forever pays.\nNow I see you in a better place (see you in a better place).\nUh."]

In [24]:
r_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                   chunk_overlap=0,
                                    # 리스트로 여러 조건을 계층적으로 설정 가능(더 큰 범위를 먼저 작성)
                                   separators=["\n\n", "\n", " "]
                                  )
r_splitter.split_text(see_you_again)

["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.",
 "We've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.",
 'When I see you again.',
 'Damn, who knew?\nAll the planes we flew, good things we been through.',
 "That I'd be standing right here talking to you.",
 "'Bout another path, I know we loved to hit the road and laugh.",
 "But something told me that it wouldn't last.",
 'Had to switch up, look at things different, see the bigger picture.',
 'Those were the days, hard work forever pays.',
 'Now I see you in a better place (see you in a better place).\nUh.']

- ChatacterTextSplitter를 사용한 결과가 만족스럽지 못하다면 더 작은 단위의 구분자를 지정해서 해결하거나 RecurciveCharacterTextSplitter를 사용하여 계층적 분할을 적용해볼 수 있음

### 웹 페이지에 접근해 쉽게 텍스트 데이터 가져오기

In [25]:
# WebBaseLoader : 웹페이지 HTML 컨텐츠를 로드해주는 클래스(내부적으로 BEautifulSoup을 사용함)
from langchain.document_loaders import WebBaseLoader
# 경고 표시는 웹 접근 시 user_agent 설정이 필요하다는 문구

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [26]:
loader = WebBaseLoader("https://news.kbs.co.kr/news/pc/view/view.do?ncd=8327815&ref=A")

# 브라우저 우회 접속
loader.requeste_kwargs = {'headers':{'User-Agent':'Mozilla/5.0'}}
# 한글 인코딩 설정
loader.encoding = 'utf-8'

docs = loader.load()

In [30]:
docs[0].page_content[:500]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKBS 뉴스\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKBS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n로그인\n회원가입\n마이페이지\nEnglish\n\n\n\n\n\n\n\n\n재난포털\n\n\n\n\n\n\n제보\n\nON AIR\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            분야별\n          \n\n\n 뉴스9 \n\n\n\n            TV 뉴스\n          \n\n\n\n            심층취재\n          \n\n\n 스포츠 \n\n\n\n            시사프로그램\n          \n\n\n\n            지역뉴스\n          \n\n\n속보\n\n\n\n          “트럼프 대통령과 25일 정상회담…업무 오찬” [지금뉴스]\n        \n\n\n\n\n\n\n\n\n\n전체메뉴\n\n\n분야별\n\n\n전체\n정치\n경제\n사회\n문화\nIT·과학\n국제\n재난·기후·환경\n생활·건강\n스포츠\n연예\n날'

#### 텍스트 데이터 정제

In [33]:
# 1. 문자열 함수 replace 활용
docs[0].page_content[:2000].replace("\n", '')

'KBS 뉴스KBS로그인회원가입마이페이지English재난포털제보ON AIR            분야별           뉴스9             TV 뉴스                      심층취재           스포츠             시사프로그램                      지역뉴스          속보          “트럼프 대통령과 25일 정상회담…업무 오찬” [지금뉴스]        전체메뉴분야별전체정치경제사회문화IT·과학국제재난·기후·환경생활·건강스포츠연예날씨이슈TV 뉴스1TV 뉴스06:00 뉴스광장 1부17:00 뉴스 507:00 뉴스광장 2부19:00 뉴스 709:30 930뉴스21:00 뉴스 912:00 뉴스 1222:50 뉴스라인 W14:00 뉴스 22TV 뉴스10:10 아침뉴스타임15:00 뉴스타임15:15 월드2418:00 경제콘서트1TV 경인뉴스광장(경인)930뉴스(경인)뉴스7(경인)뉴스9(경인)뉴스특보심층취재심층K단독탐사K제대로 보겠습니다영상K크랩글로벌K팩트체크K스포츠K취재후News Today주말엔뒷北뉴스기후는 말한다여심야심시사프로그램시사프로그램시사기획 창일요진단 라이브더 보다남북의 창특파원 보고 세계는 지금재난방송센터사사건건인사이드 경인라디오전격시사성공예감뉴스레터K지역뉴스부산순천울산전주창원대전진주청주대구충주안동춘천포항강릉광주원주목포제주바로가기좋아요 클립제보속보재난포털ON AIR뉴스 아카이브공지·정정어떤 기사를 찾으시나요?추천 인기 키워드                현재 관심이 높아지고 있는 키워드 입니다.              많이 본 뉴스1공지사항영상K8·25 한미 정상회담“트럼프 대통령과 25일 정상회담…업무 오찬” [지금뉴스]입력 2025.08.12 (11:13)수정 2025.08.12 (11:13)                        읽어주기 기능은 크롬기반의                        브라우저에서만 사용하실 수 있습니다.                    요약동영상 고정 취소  

In [34]:
# 2. 문자열 정규 표현식 사용
import re 

# r : 문자열 내 이스케이프 코드를 지칭하는 \가 있는 경우 이를 일반문자열로 인식하게 해주는 기능
re.findall(r"[\w]+", docs[0].page_content[:2000])

# 정규표현식을 사용하면 각 단어별로 따로 출력이 되기 때문에 문장으로 보고 싶으면 이를 연결시켜서 이어붙여줘야 함

['KBS',
 '뉴스',
 'KBS',
 '로그인',
 '회원가입',
 '마이페이지',
 'English',
 '재난포털',
 '제보',
 'ON',
 'AIR',
 '분야별',
 '뉴스9',
 'TV',
 '뉴스',
 '심층취재',
 '스포츠',
 '시사프로그램',
 '지역뉴스',
 '속보',
 '트럼프',
 '대통령과',
 '25일',
 '정상회담',
 '업무',
 '오찬',
 '지금뉴스',
 '전체메뉴',
 '분야별',
 '전체',
 '정치',
 '경제',
 '사회',
 '문화',
 'IT',
 '과학',
 '국제',
 '재난',
 '기후',
 '환경',
 '생활',
 '건강',
 '스포츠',
 '연예',
 '날씨',
 '이슈',
 'TV',
 '뉴스',
 '1TV',
 '뉴스',
 '06',
 '00',
 '뉴스광장',
 '1부',
 '17',
 '00',
 '뉴스',
 '5',
 '07',
 '00',
 '뉴스광장',
 '2부',
 '19',
 '00',
 '뉴스',
 '7',
 '09',
 '30',
 '930뉴스',
 '21',
 '00',
 '뉴스',
 '9',
 '12',
 '00',
 '뉴스',
 '12',
 '22',
 '50',
 '뉴스라인',
 'W',
 '14',
 '00',
 '뉴스',
 '2',
 '2TV',
 '뉴스',
 '10',
 '10',
 '아침뉴스타임',
 '15',
 '00',
 '뉴스타임',
 '15',
 '15',
 '월드24',
 '18',
 '00',
 '경제콘서트',
 '1TV',
 '경인',
 '뉴스광장',
 '경인',
 '930뉴스',
 '경인',
 '뉴스7',
 '경인',
 '뉴스9',
 '경인',
 '뉴스특보',
 '심층취재',
 '심층K',
 '단독',
 '탐사K',
 '제대로',
 '보겠습니다',
 '영상K',
 '크랩',
 '글로벌K',
 '팩트체크K',
 '스포츠K',
 '취재후',
 'News',
 'Today',
 '주말엔',
 '뒷北뉴스',
 '기후는',
 '말한다',
 '여심야심',
 '

In [35]:
" ".join(re.findall(r"[\w]+", docs[0].page_content[:2000]))

'KBS 뉴스 KBS 로그인 회원가입 마이페이지 English 재난포털 제보 ON AIR 분야별 뉴스9 TV 뉴스 심층취재 스포츠 시사프로그램 지역뉴스 속보 트럼프 대통령과 25일 정상회담 업무 오찬 지금뉴스 전체메뉴 분야별 전체 정치 경제 사회 문화 IT 과학 국제 재난 기후 환경 생활 건강 스포츠 연예 날씨 이슈 TV 뉴스 1TV 뉴스 06 00 뉴스광장 1부 17 00 뉴스 5 07 00 뉴스광장 2부 19 00 뉴스 7 09 30 930뉴스 21 00 뉴스 9 12 00 뉴스 12 22 50 뉴스라인 W 14 00 뉴스 2 2TV 뉴스 10 10 아침뉴스타임 15 00 뉴스타임 15 15 월드24 18 00 경제콘서트 1TV 경인 뉴스광장 경인 930뉴스 경인 뉴스7 경인 뉴스9 경인 뉴스특보 심층취재 심층K 단독 탐사K 제대로 보겠습니다 영상K 크랩 글로벌K 팩트체크K 스포츠K 취재후 News Today 주말엔 뒷北뉴스 기후는 말한다 여심야심 시사프로그램 시사프로그램 시사기획 창 일요진단 라이브 더 보다 남북의 창 특파원 보고 세계는 지금 재난방송센터 사사건건 인사이드 경인 라디오 전격시사 성공예감 뉴스레터K 지역뉴스 부산 순천 울산 전주 창원 대전 진주 청주 대구 충주 안동 춘천 포항 강릉 광주 원주 목포 제주 바로가기 좋아요 클립 제보 속보 재난포털 ON AIR 뉴스 아카이브 공지 정정 어떤 기사를 찾으시나요 추천 인기 키워드 현재 관심이 높아지고 있는 키워드 입니다 많이 본 뉴스 1 공지사항 영상K 8 25 한미 정상회담 트럼프 대통령과 25일 정상회담 업무 오찬 지금뉴스 입력 2025 08 12 11 13 수정 2025 08 12 11 13 읽어주기 기능은 크롬기반의 브라우저에서만 사용하실 수 있습니다 요약 동영상 고정 취소 이재명 대통령이 도널드 트럼프 미국 대통령 초청으로 미국을 방문합니다 강유정 대통령실 대변인은 오늘 12일 용산 대통령실에서 브리핑을 통해 이 대통령이 8월 25일 한미 정상회담 개최를 위해 24일부터 26일까지 미국을 방

### 문서 로더(Document Loader)
- Langchain의 문서 로더 기능을 활용해 다양한 형식의 데이터 파일들로 부터 손쉽게 텍스트를 로드할 수 있음

#### 문서 로더 종류
- TextLoader : TXT 파일 로드
- DirectoryLoader : 특정 경로에 있는 여러개의 파일 로드
- PyPDFLoader : PDF 파일 로드
- JSONLoader : JSON 파일 로드
- CSVLoader : CSV 파일 로드

### TextLoader

In [36]:
from langchain.document_loaders import TextLoader

In [39]:
loader = TextLoader("data/멜론차트 TOP100.txt", encoding='utf-8')

# load : 단일 문서로(Document객체) 반환
doc = loader.load()
doc

[Document(metadata={'source': 'data/멜론차트 TOP100.txt'}, page_content="\t타이틀\t아티스트\n0\t비의 랩소디\t임재현\n1\tTo. X\t태연 (TAEYEON)\n2\tPerfect Night\tLE SSERAFIM (르세라핌)\n3\tDrama\taespa\n4\t에피소드\t이무진\n5\t사랑은 늘 도망가\t임영웅\n6\t헤어지자 말해요\t박재정\n7\t그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))\t너드커넥션 (Nerd Connection)\n8\t인사\t범진\n9\tLove 119\tRIIZE\n10\tSeven (feat. Latto) - Clean Ver.\t정국\n11\tBaddie\tIVE (아이브)\n12\t우리들의 블루스\t임영웅\n13\t모래 알갱이\t임영웅\n14\tDo or Die\t임영웅\n15\t다시 만날 수 있을까\t임영웅\n16\t첫 눈\tEXO\n17\tI AM\tIVE (아이브)\n18\tLove Lee\tAKMU (악뮤)\n19\t이제 나만 믿어요\t임영웅\n20\tGet A Guitar\tRIIZE\n21\t무지개\t임영웅\n22\tYou & Me\t제니 (JENNIE)\n23\tLondon Boy\t임영웅\n24\t아버지\t임영웅\n25\tPolaroid\t임영웅\n26\tStanding Next to You\t정국\n27\t인생찬가\t임영웅\n28\t너의 모든 순간\t성시경\n29\t후라이의 꿈\tAKMU (악뮤)\n30\tA bientot\t임영웅\n31\tSuper Shy\tNewJeans\n32\t사막에서 꽃을 피우듯\t우디 (Woody)\n33\tDitto\tNewJeans\n34\t잘 지내자, 우리 (여름날 우리 X 로이킴)\t로이킴\n35\tETA\tNewJeans\n36\tHype Boy\tNewJeans\n37\t연애편지\t임영웅\n38\t퀸카 (Queencard)\t(여자)아이들\n39\t어떻게 이별까지 사랑하겠어, 널 

In [40]:
print(doc[0].page_content)

	타이틀	아티스트
0	비의 랩소디	임재현
1	To. X	태연 (TAEYEON)
2	Perfect Night	LE SSERAFIM (르세라핌)
3	Drama	aespa
4	에피소드	이무진
5	사랑은 늘 도망가	임영웅
6	헤어지자 말해요	박재정
7	그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))	너드커넥션 (Nerd Connection)
8	인사	범진
9	Love 119	RIIZE
10	Seven (feat. Latto) - Clean Ver.	정국
11	Baddie	IVE (아이브)
12	우리들의 블루스	임영웅
13	모래 알갱이	임영웅
14	Do or Die	임영웅
15	다시 만날 수 있을까	임영웅
16	첫 눈	EXO
17	I AM	IVE (아이브)
18	Love Lee	AKMU (악뮤)
19	이제 나만 믿어요	임영웅
20	Get A Guitar	RIIZE
21	무지개	임영웅
22	You & Me	제니 (JENNIE)
23	London Boy	임영웅
24	아버지	임영웅
25	Polaroid	임영웅
26	Standing Next to You	정국
27	인생찬가	임영웅
28	너의 모든 순간	성시경
29	후라이의 꿈	AKMU (악뮤)
30	A bientot	임영웅
31	Super Shy	NewJeans
32	사막에서 꽃을 피우듯	우디 (Woody)
33	Ditto	NewJeans
34	잘 지내자, 우리 (여름날 우리 X 로이킴)	로이킴
35	ETA	NewJeans
36	Hype Boy	NewJeans
37	연애편지	임영웅
38	퀸카 (Queencard)	(여자)아이들
39	어떻게 이별까지 사랑하겠어, 널 사랑하는 거지	AKMU (악뮤)
40	MANIAC	VIVIZ (비비지)
41	별 떨어진다 (I Do)	디오 (D.O.)
42	사랑인가 봐	멜로망스
43	Discord	QWER
44	Dynamite	방탄소년단
45	봄날	방탄소년단
46	Spicy	aespa
47	사건의 지평선	윤하 (YOUNHA)
48	사랑할 수밖에	볼빨간사춘기
49	사랑하

In [42]:
r_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                            chunk_overlap=0
                                           )

In [43]:
# load_and_split : 데이터를 로드하며 동시에 설정된 splitter를 적용해주는 함수
#                  (splitter 객체를 설정하지 않으면 로드된 문서의 page별로 Document객체로 반환됨)
docs_split1 = loader.load_and_split(r_splitter)
print(f"청크 개수 : {len(docs_split1)}개")
print()

# 각각의 청크들의 실제 텍스트를 출력
for doc in docs_split1 :
    print(doc.page_content)
    print("="*100)

청크 개수 : 5개

타이틀	아티스트
0	비의 랩소디	임재현
1	To. X	태연 (TAEYEON)
2	Perfect Night	LE SSERAFIM (르세라핌)
3	Drama	aespa
4	에피소드	이무진
5	사랑은 늘 도망가	임영웅
6	헤어지자 말해요	박재정
7	그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))	너드커넥션 (Nerd Connection)
8	인사	범진
9	Love 119	RIIZE
10	Seven (feat. Latto) - Clean Ver.	정국
11	Baddie	IVE (아이브)
12	우리들의 블루스	임영웅
13	모래 알갱이	임영웅
14	Do or Die	임영웅
15	다시 만날 수 있을까	임영웅
16	첫 눈	EXO
17	I AM	IVE (아이브)
18	Love Lee	AKMU (악뮤)
19	이제 나만 믿어요	임영웅
20	Get A Guitar	RIIZE
21	무지개	임영웅
22	You & Me	제니 (JENNIE)
23	London Boy	임영웅
24	아버지	임영웅
25	Polaroid	임영웅
26	Standing Next to You	정국
27	인생찬가	임영웅
28	너의 모든 순간	성시경
29	후라이의 꿈	AKMU (악뮤)
30	A bientot	임영웅
31	Super Shy	NewJeans
32	사막에서 꽃을 피우듯	우디 (Woody)
33	Ditto	NewJeans
34	잘 지내자, 우리 (여름날 우리 X 로이킴)	로이킴
35	ETA	NewJeans
36	Hype Boy	NewJeans
37	연애편지	임영웅
38	퀸카 (Queencard)	(여자)아이들
39	어떻게 이별까지 사랑하겠어, 널 사랑하는 거지	AKMU (악뮤)
40	MANIAC	VIVIZ (비비지)
41	별 떨어진다 (I Do)	디오 (D.O.)
42	사랑인가 봐	멜로망스
43	Discord	QWER
44	Dynamite	방탄소년단
45	봄날	방탄소년단
46	Spicy	aespa
47	사건의 지평선	윤하 (YOUNHA)
48	사랑할 수밖에	볼빨

### PyPDFLoader
- PDF파일의 텍스트 가져오기

In [44]:
# 파이썬용 PDF 처리 라이브러리
!pip install pypdf

In [45]:
from langchain.document_loaders import PyPDFLoader

In [46]:
loader = PyPDFLoader("data/기술보증기금과 한국경제.pdf")

docs = loader.load_and_split()

print("페이지 수 :", len(docs))
docs

페이지 수 : 5


[Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하

In [47]:
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=330,
                                              chunk_overlap=50,
                                              separators=["\n\n", "\n", " "]
                                             )
docs_split = loader.load_and_split(pdf_splitter)
print("청크 수 :" , len(docs_split))
print()
docs_split

청크 수 : 21



[Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은'),
 Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보